In [ ]:
import dash
from dash import html, dcc, Output, Input, State
import pandas as pd
import io
import base64

# Register the page
dash.register_page(__name__, path='/tb-tb', name="TB vs TB")

# Layout
layout = html.Div([
    html.H2("TB-TB Testing"),
    html.H4("Upload Trial Balances and General Ledger"),

    # Upload 1: Current Year TB
    html.Div([
        dcc.Upload(
            id='upload-file-1',
            children=html.Div(['Drag or Select CURRENT Year Trial Balance']),
            style={
                'width': '48%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            multiple=False
        ),
        html.Div(id='file-name-1', style={"marginLeft": "10px", "color": "green"})
    ]),

    # Upload 2: Prior Year TB
    html.Div([
        dcc.Upload(
            id='upload-file-2',
            children=html.Div(['Drag or Select PRIOR Year Trial Balance']),
            style={
                'width': '48%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            multiple=False
        ),
        html.Div(id='file-name-2', style={"marginLeft": "10px", "color": "green"})
    ]),

    # Upload 3: General Ledger
    html.Div([
        dcc.Upload(
            id='upload-file-3',
            children=html.Div(['Drag or Select GENERAL LEDGER']),
            style={
                'width': '48%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            multiple=False
        ),
        html.Div(id='file-name-3', style={"marginLeft": "10px", "color": "green"})
    ]),

    html.Button("Download Result", id="download-btn", n_clicks=0, disabled=True, style={"marginTop": "20px"}),

    # Loading spinner and status message
    dcc.Loading(
        id="loading-spinner",
        type="default",
        children=html.Div(id="download-status", style={"marginTop": "10px", "color": "#0074D9"})
    ),

    dcc.Download(id="download-excel")
])

# Display filenames after upload
@dash.callback(
    Output('file-name-1', 'children'),
    Input('upload-file-1', 'filename'),
    prevent_initial_call=True
)
def update_filename1(name):
    return f"✅ Uploaded: {name}" if name else ""

@dash.callback(
    Output('file-name-2', 'children'),
    Input('upload-file-2', 'filename'),
    prevent_initial_call=True
)
def update_filename2(name):
    return f"✅ Uploaded: {name}" if name else ""

@dash.callback(
    Output('file-name-3', 'children'),
    Input('upload-file-3', 'filename'),
    prevent_initial_call=True
)
def update_filename3(name):
    return f"✅ Uploaded: {name}" if name else ""

# Enable download button only when all files are uploaded
@dash.callback(
    Output("download-btn", "disabled"),
    Input("upload-file-1", "contents"),
    Input("upload-file-2", "contents"),
    Input("upload-file-3", "contents"),
    prevent_initial_call=True
)
def toggle_download_button(file1, file2, file3):
    return not all([file1, file2, file3])

# Generate download file and show status
@dash.callback(
    Output("download-excel", "data"),
    Output("download-status", "children"),
    Input("download-btn", "n_clicks"),
    State("upload-file-1", "contents"),
    State("upload-file-2", "contents"),
    State("upload-file-3", "contents"),
    prevent_initial_call=True
)
def generate_excel(n_clicks, curr_tb_content, prior_tb_content, gl_content):
    def parse_contents(contents):
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        return pd.read_excel(io.BytesIO(decoded), engine="openpyxl")

    if not all([curr_tb_content, prior_tb_content, gl_content]):
        return None, "❌ Please upload all three files before downloading."

    try:
        curr_tb = parse_contents(curr_tb_content)
        prior_tb = parse_contents(prior_tb_content)
        gl = parse_contents(gl_content)

        output = io.BytesIO()
        with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
            curr_tb.to_excel(writer, index=False, sheet_name='Current TB')
            prior_tb.to_excel(writer, index=False, sheet_name='Prior TB')
            gl.to_excel(writer, index=False, sheet_name='General Ledger')
        output.seek(0)

        return dcc.send_bytes(output.getvalue(), filename="result.xlsx"), "✅ Excel file ready for download."

    except Exception as e:
        return None, f"❌ Error: {str(e)}"


In [ ]:
import dash
from dash import html, dcc, Output, Input, State
import pandas as pd
import io
import base64

# Register the page
dash.register_page(__name__, path='/tb-tb', name="TB vs TB")

# Layout
layout = html.Div([
    html.H2("TB-TB Testing"),
    html.H4("Upload Trial Balances and General Ledger"),

    # Upload 1: Current Year TB
    html.Div([
        dcc.Upload(
            id='upload-file-1',
            children=html.Div(['Drag or Select CURRENT Year Trial Balance']),
            style={
                'width': '48%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            multiple=False
        ),
        html.Div(id='file-name-1', style={"marginLeft": "10px", "color": "green"})
    ]),

    # Upload 2: Prior Year TB
    html.Div([
        dcc.Upload(
            id='upload-file-2',
            children=html.Div(['Drag or Select PRIOR Year Trial Balance']),
            style={
                'width': '48%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            multiple=False
        ),
        html.Div(id='file-name-2', style={"marginLeft": "10px", "color": "green"})
    ]),

    # Upload 3: General Ledger
    html.Div([
        dcc.Upload(
            id='upload-file-3',
            children=html.Div(['Drag or Select GENERAL LEDGER']),
            style={
                'width': '48%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            multiple=False
        ),
        html.Div(id='file-name-3', style={"marginLeft": "10px", "color": "green"})
    ]),

    # Column Mapping UI (Visible after file uploads)
    html.Div(id="column-mapping-1"),
    html.Div(id="column-mapping-2"),
    html.Div(id="column-mapping-3"),

    html.Button("Download Result", id="download-btn", n_clicks=0, disabled=True, style={"marginTop": "20px"}),

    # Loading spinner and status message
    dcc.Loading(
        id="loading-spinner",
        type="default",
        children=html.Div(id="download-status", style={"marginTop": "10px", "color": "#0074D9"})
    ),

    dcc.Download(id="download-excel")
])

# Display filenames after upload
@dash.callback(
    Output('file-name-1', 'children'),
    Input('upload-file-1', 'filename'),
    prevent_initial_call=True
)
def update_filename1(name):
    return f"✅ Uploaded: {name}" if name else ""

@dash.callback(
    Output('file-name-2', 'children'),
    Input('upload-file-2', 'filename'),
    prevent_initial_call=True
)
def update_filename2(name):
    return f"✅ Uploaded: {name}" if name else ""

@dash.callback(
    Output('file-name-3', 'children'),
    Input('upload-file-3', 'filename'),
    prevent_initial_call=True
)
def update_filename3(name):
    return f"✅ Uploaded: {name}" if name else ""

# Enable download button only when all files are uploaded
@dash.callback(
    Output("download-btn", "disabled"),
    Input("upload-file-1", "contents"),
    Input("upload-file-2", "contents"),
    Input("upload-file-3", "contents"),
    prevent_initial_call=True
)
def toggle_download_button(file1, file2, file3):
    return not all([file1, file2, file3])

# Display column mappings for each file after upload
@dash.callback(
    Output("column-mapping-1", "children"),
    Input('upload-file-1', 'contents'),
    prevent_initial_call=True
)
def display_column_mapping_1(contents):
    if contents is None:
        return ""
    df = pd.read_excel(io.BytesIO(base64.b64decode(contents.split(',')[1])), engine='openpyxl')
    columns = df.columns.tolist()
    return html.Div([
        html.H5("Map Columns for Current Year Trial Balance"),
        dcc.Dropdown(id='account-code-dropdown-1', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Account Code"),
        dcc.Dropdown(id='account-name-dropdown-1', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Account Name"),
        dcc.Dropdown(id='amount-dropdown-1', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Amount")
    ])

@dash.callback(
    Output("column-mapping-2", "children"),
    Input('upload-file-2', 'contents'),
    prevent_initial_call=True
)
def display_column_mapping_2(contents):
    if contents is None:
        return ""
    df = pd.read_excel(io.BytesIO(base64.b64decode(contents.split(',')[1])), engine='openpyxl')
    columns = df.columns.tolist()
    return html.Div([
        html.H5("Map Columns for Prior Year Trial Balance"),
        dcc.Dropdown(id='account-code-dropdown-2', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Account Code"),
        dcc.Dropdown(id='account-name-dropdown-2', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Account Name"),
        dcc.Dropdown(id='amount-dropdown-2', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Amount")
    ])

@dash.callback(
    Output("column-mapping-3", "children"),
    Input('upload-file-3', 'contents'),
    prevent_initial_call=True
)
def display_column_mapping_3(contents):
    if contents is None:
        return ""
    df = pd.read_excel(io.BytesIO(base64.b64decode(contents.split(',')[1])), engine='openpyxl')
    columns = df.columns.tolist()
    return html.Div([
        html.H5("Map Columns for General Ledger"),
        dcc.Dropdown(id='account-code-dropdown-3', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Account Code"),
        dcc.Dropdown(id='account-name-dropdown-3', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Account Name"),
        dcc.Dropdown(id='amount-dropdown-3', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Amount")
    ])

# Generate download file and show status
@dash.callback(
    Output("download-excel", "data"),
    Output("download-status", "children"),
    Input("download-btn", "n_clicks"),
    State("upload-file-1", "contents"),
    State("upload-file-2", "contents"),
    State("upload-file-3", "contents"),
    State('account-code-dropdown-1', 'value'),
    State('account-name-dropdown-1', 'value'),
    State('amount-dropdown-1', 'value'),
    State('account-code-dropdown-2', 'value'),
    State('account-name-dropdown-2', 'value'),
    State('amount-dropdown-2', 'value'),
    State('account-code-dropdown-3', 'value'),
    State('account-name-dropdown-3', 'value'),
    State('amount-dropdown-3', 'value'),
    prevent_initial_call=True
)
def generate_excel(n_clicks, curr_tb_content, prior_tb_content, gl_content,
                   curr_account_code, curr_account_name, curr_amount,
                   prior_account_code, prior_account_name, prior_amount,
                   gl_account_code, gl_account_name, gl_amount):

    def parse_contents(contents):
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        return pd.read_excel(io.BytesIO(decoded), engine="openpyxl")

    # Ensure all files and mappings are provided
    if not all([curr_tb_content, prior_tb_content, gl_content,
                curr_account_code, curr_account_name, curr_amount,
                prior_account_code, prior_account_name, prior_amount,
                gl_account_code, gl_account_name, gl_amount]):
        return None, "❌ Please upload all files and map the columns before downloading."

    try:
        # Parse contents and map columns
        curr_tb = parse_contents(curr_tb_content)
        prior_tb = parse_contents(prior_tb_content)
        gl = parse_contents(gl_content)

        # Mapping for current year TB
        curr_tb = curr_tb.rename(columns={
            curr_account_code: "ACCOUNT CODE",
            curr_account_name: "ACCOUNT NAME",
            curr_amount: "AMOUNT"
        })

        # Mapping for prior year TB
        prior_tb = prior_tb.rename(columns={
            prior_account_code: "ACCOUNT CODE",
            prior_account_name: "ACCOUNT NAME",
            prior_amount: "AMOUNT"
        })

        # Mapping for GL
        gl = gl.rename(columns={
            gl_account_code: "ACCOUNT CODE",
            gl_account_name: "ACCOUNT NAME",
            gl_amount: "AMOUNT"
        })

        # Output to Excel
        output = io.BytesIO()
        with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
            curr_tb.to_excel(writer, index=False, sheet_name='Current TB')
            prior_tb.to_excel(writer, index=False, sheet_name='Prior TB')
            gl.to_excel(writer, index=False, sheet_name='General Ledger')
        output.seek(0)

        return dcc.send_bytes(output.getvalue(), filename="result.xlsx"), "✅ Excel file ready for download."

    except Exception as e:
        return None, f"❌ Error: {str(e)}"


In [ ]:
import dash
from dash import html, dcc, Output, Input, State
import pandas as pd
import io
import base64

# Register the page
dash.register_page("Inventory-Completeness", path="/inventory")

layout = html.Div([
    html.H2("Inventory Completeness Testing"),
    html.H4("Upload Inventory Reports"),

    # Upload 1: Current Year Inventory
    html.Div([
        dcc.Upload(
            id='upload-inventory-1',
            children=html.Div(['Drag or Select CURRENT Year Inventory Report']),
            style={
                'width': '48%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            multiple=False
        ),
        html.Div(id='inventory-name-1', style={"marginLeft": "10px", "color": "green"})
    ]),

    # Upload 2: Prior Year Inventory
    html.Div([
        dcc.Upload(
            id='upload-inventory-2',
            children=html.Div(['Drag or Select PRIOR Year Inventory Report']),
            style={
                'width': '48%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            multiple=False
        ),
        html.Div(id='inventory-name-2', style={"marginLeft": "10px", "color": "green"})
    ]),

    # Upload 3: Movement Report
    html.Div([
        dcc.Upload(
            id='upload-inventory-3',
            children=html.Div(['Drag or Select MOVEMENT Report']),
            style={
                'width': '48%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            multiple=False
        ),
        html.Div(id='inventory-name-3', style={"marginLeft": "10px", "color": "green"})
    ]),

    # Column Mapping UI (Visible after file uploads)
    html.Div(id="inventory-column-mapping-1"),
    html.Div(id="inventory-column-mapping-2"),
    html.Div(id="inventory-column-mapping-3"),

    html.Button("Download Result", id="inventory-download-btn", n_clicks=0, disabled=True, style={"marginTop": "20px"}),

    # Loading spinner and status message
    dcc.Loading(
        id="inventory-loading-spinner",
        type="default",  # Show default spinner
        children=html.Div(id="inventory-download-status", style={"marginTop": "10px", "color": "#0074D9"})
    ),

    dcc.Download(id="inventory-download-excel")
])

# File name displays
@dash.callback(
    Output('inventory-name-1', 'children'),
    Input('upload-inventory-1', 'filename'),
    prevent_initial_call=True
)
def update_inventory_name1(name):
    return f"✅ Uploaded: {name}" if name else ""

@dash.callback(
    Output('inventory-name-2', 'children'),
    Input('upload-inventory-2', 'filename'),
    prevent_initial_call=True
)
def update_inventory_name2(name):
    return f"✅ Uploaded: {name}" if name else ""

@dash.callback(
    Output('inventory-name-3', 'children'),
    Input('upload-inventory-3', 'filename'),
    prevent_initial_call=True
)
def update_inventory_name3(name):
    return f"✅ Uploaded: {name}" if name else ""

# Enable download button only when all files are uploaded
@dash.callback(
    Output("inventory-download-btn", "disabled"),
    Input("upload-inventory-1", "contents"),
    Input("upload-inventory-2", "contents"),
    Input("upload-inventory-3", "contents"),
    prevent_initial_call=True
)
def toggle_download_button(file1, file2, file3):
    return not all([file1, file2, file3])

# Display column mappings for each file after upload
@dash.callback(
    Output("inventory-column-mapping-1", "children"),
    Input('upload-inventory-1', 'contents'),
    prevent_initial_call=True
)
def display_column_mapping_1(contents):
    if contents is None:
        return ""
    df = pd.read_excel(io.BytesIO(base64.b64decode(contents.split(',')[1])), engine='openpyxl')
    columns = df.columns.tolist()
    return html.Div([
        html.H5("Map Columns for Current Year Inventory"),
        dcc.Dropdown(id='item-code-dropdown-1', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Item Code"),
        dcc.Dropdown(id='item-name-dropdown-1', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Item Name"),
        dcc.Dropdown(id='quantity-dropdown-1', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Quantity")
    ])

@dash.callback(
    Output("inventory-column-mapping-2", "children"),
    Input('upload-inventory-2', 'contents'),
    prevent_initial_call=True
)
def display_column_mapping_2(contents):
    if contents is None:
        return ""
    df = pd.read_excel(io.BytesIO(base64.b64decode(contents.split(',')[1])), engine='openpyxl')
    columns = df.columns.tolist()
    return html.Div([
        html.H5("Map Columns for Prior Year Inventory"),
        dcc.Dropdown(id='item-code-dropdown-2', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Item Code"),
        dcc.Dropdown(id='item-name-dropdown-2', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Item Name"),
        dcc.Dropdown(id='quantity-dropdown-2', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Quantity")
    ])

@dash.callback(
    Output("inventory-column-mapping-3", "children"),
    Input('upload-inventory-3', 'contents'),
    prevent_initial_call=True
)
def display_column_mapping_3(contents):
    if contents is None:
        return ""
    df = pd.read_excel(io.BytesIO(base64.b64decode(contents.split(',')[1])), engine='openpyxl')
    columns = df.columns.tolist()
    return html.Div([
        html.H5("Map Columns for Movement Report"),
        dcc.Dropdown(id='item-code-dropdown-3', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Item Code"),
        dcc.Dropdown(id='item-name-dropdown-3', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Item Name"),
        dcc.Dropdown(id='quantity-dropdown-3', options=[{'label': col, 'value': col} for col in columns], placeholder="Select Quantity")
    ])

# Generate download file and show status
@dash.callback(
    Output("inventory-download-excel", "data"),
    Output("inventory-download-status", "children"),
    Input("inventory-download-btn", "n_clicks"),
    State("upload-inventory-1", "contents"),
    State("upload-inventory-2", "contents"),
    State("upload-inventory-3", "contents"),
    State('item-code-dropdown-1', 'value'),
    State('item-name-dropdown-1', 'value'),
    State('quantity-dropdown-1', 'value'),
    State('item-code-dropdown-2', 'value'),
    State('item-name-dropdown-2', 'value'),
    State('quantity-dropdown-2', 'value'),
    State('item-code-dropdown-3', 'value'),
    State('item-name-dropdown-3', 'value'),
    State('quantity-dropdown-3', 'value'),
    prevent_initial_call=True
)
def generate_inventory_excel(n_clicks, curr_inv_content, prior_inv_content, move_content,
                             curr_item_code, curr_item_name, curr_quantity,
                             prior_item_code, prior_item_name, prior_quantity,
                             move_item_code, move_item_name, move_quantity):

    def parse_contents(contents):
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        return pd.read_excel(io.BytesIO(decoded), engine="openpyxl")

    # Ensure all files and mappings are provided
    if not all([curr_inv_content, prior_inv_content, move_content,
                curr_item_code, curr_item_name, curr_quantity,
                prior_item_code, prior_item_name, prior_quantity,
                move_item_code, move_item_name, move_quantity]):
        return None, "❌ Please upload all files and map columns before downloading."

    try:
        curr_inv = parse_contents(curr_inv_content)
        prior_inv = parse_contents(prior_inv_content)
        movement = parse_contents(move_content)

        # Apply mappings
        curr_inv = curr_inv.rename(columns={curr_item_code: "ITEM CODE",
                                            curr_item_name: "ITEM NAME",
                                            curr_quantity: "QUANTITY"})
        prior_inv = prior_inv.rename(columns={prior_item_code: "ITEM CODE",
                                              prior_item_name: "ITEM NAME",
                                              prior_quantity: "QUANTITY"})
        movement = movement.rename(columns={move_item_code: "ITEM CODE",
                                            move_item_name: "ITEM NAME",
                                            move_quantity: "QUANTITY"})

        output = io.BytesIO()
        with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
            curr_inv.to_excel(writer, index=False, sheet_name='Current Inventory')
            prior_inv.to_excel(writer, index=False, sheet_name='Prior Inventory')
            movement.to_excel(writer, index=False, sheet_name='Movement Report')
        output.seek(0)

        return dcc.send_bytes(output.getvalue(), filename="inventory_result.xlsx"), "✅ Excel file ready for download."

    except Exception as e:
        return None, f"❌ Error: {str(e)}"


In [ ]:
import dash
from dash import html, dcc, Output, Input, State, callback_context
import pandas as pd
import io
import base64
import difflib

# ✅ Register this page with the correct name and path
dash.register_page(
    __name__,  # ✅ Use __name__ here
    path="/gl_mapping",
    name="GL Mapping"
)

# ✅ Define layout as a variable named layout
layout = html.Div([
    html.H2("Upload General Ledger and Map Columns"),

    # Upload: General Ledger
    html.Div([
        dcc.Upload(
            id='upload-gl',
            children=html.Div(['Drag or Select General Ledger']),
            style={
                'width': '48%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            multiple=False
        ),
        html.Div(id='gl-file-name', style={"marginLeft": "10px", "color": "green"})
    ]),

    html.Div(id="column-mapping", style={"marginTop": "20px"}),

    html.Button("Download GL", id="gl-download-btn", n_clicks=0, style={"marginTop": "20px"}, disabled=True),

    dcc.Loading(
        id="gl-loading-spinner",
        type="default",
        children=html.Div(id="gl-download-status", style={"marginTop": "10px", "color": "#0074D9"})
    ),

    dcc.Download(id="gl-download-excel")
])

# ✅ Callback to show uploaded file name
@dash.callback(
    Output('gl-file-name', 'children', allow_duplicate=True),
    Input('upload-gl', 'filename'),
    prevent_initial_call=True
)
def update_gl_filename(name):
    return f"✅ Uploaded: {name}" if name else ""

# ✅ Callback to detect columns and create dropdowns
@dash.callback(
    Output('column-mapping', 'children'),
    Output("gl-download-btn", "disabled"),
    Input('upload-gl', 'contents'),
    prevent_initial_call=True
)
def generate_column_mapping(gl_content):
    def parse_contents(contents):
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        return pd.read_excel(io.BytesIO(decoded), engine="openpyxl")

    try:
        gl_df = parse_contents(gl_content)
        detected_columns = gl_df.columns.tolist()

        required_columns = [
            "ACCOUNT CODE", "ACCOUNT NAME", "TRANSACTION DATE",
            "TRANSACTION SOURCE", "LEAD SHEET NUMBER", "AMOUNT"
        ]

        dropdowns = []
        for col in required_columns:
            # Fuzzy match
            closest_match = difflib.get_close_matches(col, detected_columns, n=1, cutoff=0.5)
            preselected = closest_match[0] if closest_match else None

            dropdowns.append(html.Div([
                html.Label(f"Select column for: {col}"),
                dcc.Dropdown(
                    id=f"dropdown-{col}",
                    options=[{'label': c, 'value': c} for c in detected_columns],
                    placeholder=f"Select {col}",
                    value=preselected,
                    style={"width": "50%"}
                )
            ], style={"marginBottom": "20px"}))

        return dropdowns, False

    except Exception as e:
        return html.Div([f"❌ Error: {str(e)}"]), True

# ✅ Download button logic
@dash.callback(
    Output("gl-download-excel", "data", allow_duplicate=True),
    Output("gl-download-status", "children", allow_duplicate=True),
    Input("gl-download-btn", "n_clicks"),
    State("upload-gl", "contents"),
    State("dropdown-ACCOUNT CODE", "value"),
    State("dropdown-ACCOUNT NAME", "value"),
    State("dropdown-TRANSACTION DATE", "value"),
    State("dropdown-TRANSACTION SOURCE", "value"),
    State("dropdown-LEAD SHEET NUMBER", "value"),
    State("dropdown-AMOUNT", "value"),
    prevent_initial_call=True
)
def generate_gl_excel(n_clicks, gl_content, account_code_col, account_name_col, txn_date_col, txn_source_col,
                      lead_sheet_col, amount_col):
    ctx = callback_context
    if not ctx.triggered or ctx.triggered[0]['prop_id'].split('.')[0] != "gl-download-btn":
        return dash.no_update, dash.no_update

    def parse_contents(contents):
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        return pd.read_excel(io.BytesIO(decoded), engine="openpyxl")

    try:
        gl_df = parse_contents(gl_content)

        if not all([account_code_col, account_name_col, txn_date_col, txn_source_col, lead_sheet_col, amount_col]):
            return None, "❌ Please select all required fields"

        mapped_df = gl_df.rename(columns={
            account_code_col: "ACCOUNT CODE",
            account_name_col: "ACCOUNT NAME",
            txn_date_col: "TRANSACTION DATE",
            txn_source_col: "TRANSACTION SOURCE",
            lead_sheet_col: "LEAD SHEET NUMBER",
            amount_col: "AMOUNT"
        })

        output = io.BytesIO()
        with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
            mapped_df.to_excel(writer, index=False, sheet_name='General Ledger')
        output.seek(0)

        return dcc.send_bytes(output.getvalue(), filename="mapped_general_ledger.xlsx"), \
               "✅ GL file with mapped columns ready for download."

    except Exception as e:
        return None, f"❌ Error: {str(e)}"
